<a href="https://colab.research.google.com/github/danilorogerio/Projeto_Imers-o-Alura-Google/blob/main/Projeto_Imersao_Alura_Google_GeminAI_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                   **GeminAIQuiz - Quiz Dinâmico com Google Gemini**

**Descrição do Projeto:**

Projeto de quiz dinâmico utilizando a API do Google Gemini. O Gemini será responsável por gerar as perguntas, as alternativas de resposta, avaliar as respostas do usuário e fornecer feedback, tornando o processo de aprendizado mais interativo e personalizado.

**Funcionalidades:**

Geração de Perguntas e Respostas: O usuário poderá escolher um tema ou área de conhecimento para o quiz. O Gemini gerará perguntas, juntamente com 5 alternativas de resposta (A, B, C, D, E).

Avaliação de Respostas: O Gemini avaliará as respostas fornecidas pelo usuário, comparando-as com a resposta correta.Feedback poderá ser fornecido, indicando se a resposta está correta ou incorreta, além de explicar o porquê.

Consolidação de Resultados: Ao final do quiz, o Gemini fornecerá um resumo do desempenho do usuário, incluindo Número de perguntas respondidas corretamente, porcentagem de acerto.

**Implementação:**

Linguagem de Programação: Python.

Plataforma: Google Colab.

API: Google Gemini.

Interface do Usuário:A interface do quiz será implementada no Google Colab, utilizando células de código e texto para exibir as perguntas, receber as respostas do usuário e apresentar os resultados. Elementos visuais, como cores e formatação, serão utilizados para tornar o quiz mais atraente e intuitivo.

**Fluxo do Programa:**

Inicialização: O usuário define o tema.

Loop Principal:O Gemini gera uma pergunta e suas alternativas de resposta. As informações são exibidas ao usuário. O usuário fornece a resposta. O Gemini avalia a resposta e fornece feedback.
O processo se repete até o usuário optar por sair.

Consolidação de Resultados: O Gemini calcula e apresenta o desempenho do usuário.

**Benefícios:**

Aprendizado Personalizado: O Gemini adapta as perguntas ao nível de conhecimento do usuário.

Feedback Detalhado: O usuário recebe explicações sobre as respostas, aprofundando a compreensão do tema.

Engajamento Aprimorado: A geração dinâmica de perguntas torna o quiz mais interessante e desafiador.

**Melhorias**

-Foi identificado inconsistências na consolidação dos resultados quando ficou a cargo somente do Gemini, provavelmente solucionável com refino do prompt. A solução aplicada neste projeto foi fazer parsing do histórico de chat e informar ao Gemini da quantidade de erros e acertos mediante a função *obter_resultado*.

-Foi percebido baixa aleatoridade nas perguntas quando se revisita um mesmo tema o que provavelmente possa ser resolvido com ajuste dos parametros do modelo.

-Poucos testes realizados

**Conclusão:**
Este projeto demonstra o potencial do Google Gemini para criar experiências de aprendizado inovadoras e personalizadas. A capacidade do Gemini de gerar conteúdo dinâmico, avaliar respostas e fornecer feedback detalhado o torna uma ferramenta poderosa para o desenvolvimento de quizzes interativos e eficazes.


** descrição de projeto gerada com auxílio do  Google AI Studio *


In [16]:
from google.colab import userdata
import google.generativeai as genai

In [17]:
genai.configure(api_key= userdata.get('GOOGLE_API_KEY'))

Funções para layout da tela:
- print_titulo_quiz : gerado pelo Google AI Studio
- aplicar_estilo
- borda_quadrada : gerado pelo Google AI Studio
Documentação gerada pelo Google AI Studio

In [18]:
def print_titulo_quiz(titulo):
  """Imprime um título de jogo estilo quiz com borda."""

  borda = "*" * (len(titulo) + 4)
  print(borda)
  print(f"* {titulo} *")
  print(borda)

In [19]:
def aplicar_estilo(texto, estilo):
  """Aplica um estilo de formatação ANSI a uma string.

  Args:
    texto: A string a ser formatada.
    estilo: O estilo a ser aplicado. As opções são: "negrito", "azul", "vermelho".

  Returns:
    A string formatada com o estilo especificado.

  Raises:
    ValueError: Se o estilo fornecido não for válido.
  """
  if estilo == "negrito":
    return f"\033[1m{texto}\033[0m"
  elif estilo == "azul":
    return f"\033[34m{texto}\033[0m"
  elif estilo == "vermelho":
    return f"\033[91m{texto}\033[0m"

In [20]:
def borda_quadrada(texto):
  """Adiciona uma borda quadrada em torno de uma string.

  Args:
    texto: A string a ser enquadrada.

  Returns:
    A string com a borda quadrada.
  """

  linhas = texto.splitlines()
  largura_maxima = max(len(linha) for linha in linhas)

  borda_horizontal = "+" + "-" * (largura_maxima + 2) + "+"
  texto_enquadrado = borda_horizontal + "\n"

  for linha in linhas:
    texto_enquadrado += "| " + linha.ljust(largura_maxima) + " |\n"

  texto_enquadrado += borda_horizontal

  return texto_enquadrado


Funções definidas para template do prompt.
- template_pergunta
- template_resposta
- template_resultado

Documentação do código gerada pelo Google AI Studio com pequenos ajustes.

In [21]:
def template_pergunta(assunto):
  """Gera um template de instrução para uo Gemini criar uma pergunta sobre um assunto específico.

  A função recebe um assunto como entrada e cria uma string de instrução detalhada para o Gemini. A instrução solicita ao modelo que gere uma pergunta
  sobre o assunto fornecido, com cinco alternativas de resposta (a-e).

  Args:
    assunto: O tema sobre o qual a pergunta deve ser elaborada.

  Returns:
    Uma string contendo a instrução formatada para o modelo. A instrução inclui:
      * A instrução para gerar uma pergunta sobre o assunto.
      * A instrução para fornecer cinco alternativas de resposta (a-e).
      * A instrução para formatar a pergunta e as alternativas de acordo com um padrão específico.
      * A instrução para indicar explicitamente que não possui conhecimento sobre o assunto, caso ele seja desconhecido.

  """
  Instrucoes_Pergunta_Modelo = f"""
  Crie uma pergunta sobre o tópico '{assunto}' com cinco alternativas e somente uma resposta correta.
  Use a formatação abaixo:

  a) Resposta 1
  b) Resposta 2
  c) Resposta 3
  d) Resposta 4
  e) Resposta 5.

  max_tokens: 50,

  """
  return Instrucoes_Pergunta_Modelo

def template_resposta(prompt):
  """Cria um template de instrução para avaliar a resposta de um jogador.

  A função recebe a alternativa selecionada pelo jogador e gera um texto
  que instrui o Gemini a avaliar a correção da resposta.

  Args:
    prompt: A alternativa selecionada pelo jogador (e.g., "A", "B", "C", etc.).

  Returns:
    Uma string contendo a instrução formatada para o modelo. A instrução
    informa ao modelo qual alternativa o jogador escolheu e solicita que ele
    avalie se a escolha está correta ou não.
  """
  Instrucoes_Resposta_Modelo = f"""
  O jogador selecionou alternativa {prompt}. Avalie se a resposta está certa ou errada.
  max_tokens: 150,

  """
  return Instrucoes_Resposta_Modelo


In [22]:
def template_resultado(historico, certas, erradas):

    Instruções_Resultado = f"""
    Analise o histórico de chat {historico} e informe o número de respostas {certas} e {erradas}.

    Responda no seguinte formato:
    Respostas certas: {certas}
    Respostas erradas: {erradas}

    Sugira conselhos de estudo se {erradas} > {certas}
    Elogie se  {certas} >= {erradas}

    """
    return Instruções_Resultado

Funções de validação:
- gemini_conhece_tema
- avalia_resposta
- seleciona_alternativa
- tratar_pergunta
- obter_resultados

Documentação gerada pelo Google AI Studio

In [23]:
def gemini_conhece_tema(pergunta):
  """Verifica se a resposta gerada pelo modelo Gemini indica conhecimento sobre o assunto.

  A função analisa os primeiros 25 caracteres da resposta (convertidos para maiúsculas)
  e busca por termos que indiquem falta de conhecimento, como "Não sei", "Não conheço", etc.

  Args:
    pergunta_gerada: A string contendo a resposta gerada pelo modelo Gemini.

  Returns:
    True se a resposta NÃO contiver termos que indiquem falta de conhecimento,
    sugerindo que o modelo POSSUI conhecimento sobre o assunto.
    False se a resposta contiver algum termo que indique falta de conhecimento,
    sugerindo que o modelo NÃO possui conhecimento sobre o assunto.
  """
  string = pergunta[0:30].upper()
  lista_termos = ["NÃO SEI", "NÃO CONHEÇO", "NÃO TENHO CONHECIMENTO","NÃO É POSSÍVEL"]
  for termo in lista_termos:
    if termo in string:
      return False
  return True

In [24]:
def avalia_resposta(resposta):
  """
  Verifica se a resposta fornecida contém termos negativos, indicando uma resposta incorreta.

  A função analisa os primeiros 30 caracteres da resposta em maiúsculo, buscando a
  presença de termos negativos como "ERR", "INCORR" ou "INDEV". Se algum desses termos
  for encontrado na resposta, a função considera a resposta como incorreta e retorna False.
  Caso contrário, a resposta é considerada correta e a função retorna True.

  Args:
    resposta (str): A resposta a ser avaliada.

  Returns:
    bool: True se a resposta for considerada correta (não contém termos negativos), False caso contrário.
  """
  string = resposta[0:30].upper()
  lista_termos = ["ERR", "INCORRE", "INDEV"]
  for termo in lista_termos:
    if termo in string:
      return False
  return True

In [25]:
def seleciona_alternativa():
  """
  Solicita ao usuário que insira uma alternativa válida (A, B, C, D ou E).

  A função exibe um menu com uma linha horizontal, solicita a entrada do usuário
  para a alternativa correta e verifica se a entrada é válida (A, B, C, D ou E).
  Se a entrada não for válida, exibe uma mensagem de erro e solicita
  novamente a entrada do usuário. O loop continua até que uma alternativa válida
  seja inserida.

  Returns:
    str: A alternativa válida inserida pelo usuário em maiúsculo (A, B, C, D ou E).
  """
  print("-" * 100)
  while True:
    alternativa = input("Qual a alternativa correta:\n")
    if alternativa.upper() in ("A", "B","C", "D", "E"):
      break
    else:
      print("Essa alternativa não existe! Digite a, b, c, d ou e")
  print("-" * 100)
  return alternativa

O Gemini para algumas situações tem gerado a pergunta seguida da resposta correta no formato ** {Resposta correta} **. Assim utilizei essa função para fazer o tratamento.

In [26]:
def tratar_pergunta(string):
  """
  Remove todos os caracteres após os caracteres '**' em uma string.

  A função divide a string nos caracteres '**' e retorna a parte da string antes
  da ocorrência de '**'.

  Args:
    string: A string a ser processada.

  Returns:
    str: A parte da string antes dos caracteres '**', ou a string original
         caso '**' não esteja presente.
  """
  partes = string.split("**")
  return partes[0]

In [27]:
def obter_resultado(history):
  """
  Analisa o histórico de conversas do chatbot para determinar o número de respostas corretas e incorretas.

  Args:
      history: Uma lista de objetos representando o histórico de conversas do chatbot.

  Returns:
      Uma tupla contendo o número de respostas corretas e o número de respostas incorretas.
  """
  respostas_corretas = 0
  respostas_incorretas = 0
  for i in range (3, len(history), 4):
    string = str(chat.history[i].parts).split(".")[0].upper()
    if string.find("CERT", 0, 25) > 0:
      respostas_corretas += 1
    if string.find("ERRA", 0, 25) > 0:
      respostas_incorretas += 1
  return (respostas_corretas, respostas_incorretas)

Configuração dos parâmetros do modelo, ensinadas durante a imersão.

In [28]:
generation_config = {
  "temperature": 0.3,
  "top_p": 0.9,
  "top_k": 10,
  "candidate_count": 1,
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

Cria o modelo com as configurações acima

In [29]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

Corpo principal do Quiz onde é executado um chat com o Gemini cujo funcionamento é o seguinte:
- Gemini gera perguntas de um assunto escolhido pelo jogador;
- Jogador escolhe uma alternativa;
- Gemini analisa a resposta e dá uma devolutiva ao jogador;
- Jogador pode finalizar ou prosseguir para próxima pergunta. Caso encerre, o Gemini consolida as respostas corretas e incorretas e mostra um resultado.

In [30]:
# Cria a tela inicial
print_titulo_quiz("GeminAIQuiz - Quiz Dinâmico com Google Gemini")

# Inicia o chat com o Gemini
chat = model.start_chat(history=[])

#Usuario escolhe o assunto do Quiz
tema = input("Escolha um tema para o Quiz (Tema sugerido Python): \n")
print("")
print("-" * 100)
prompt = ""
pergunta = ""

while True:
    # Gemini gera a pergunta conforme assunto escolhido e formatação de exemplo
    pergunta_modelo = chat.send_message(template_pergunta(tema))

    # Caso o tema seja conhecido pelo Gemini a pergunta será apresentada
    if gemini_conhece_tema(pergunta_modelo.text):
      pergunta = tratar_pergunta(pergunta_modelo.text)
      print(aplicar_estilo(pergunta, "negrito"), "\n")
    else:
      break

    # jogador escolhe alternativa
    prompt = seleciona_alternativa()

    # Gemini avalia a resposta
    resposta_modelo = chat.send_message(template_resposta(prompt))

    # printa na tela a resposta do GEMINI e aplica cor azul se correta e vermelha se incorreta
    if  avalia_resposta(resposta_modelo.text):
      print("\n", aplicar_estilo(resposta_modelo.text, "azul"), "\n")
    else:
       print("\n", aplicar_estilo(resposta_modelo.text, "vermelho"), "\n")

    # Opção para finalizar o jogo ou continuar
    sair_jogo = input("Digite <S> para finalizar o jogo ou qualquer outra tecla para continuar \n ")
    if sair_jogo.upper() == "S":
      break

# Solicita ao Gemini para avaliar o resultado das respostas
if gemini_conhece_tema(pergunta_modelo.text):
  num_certas_erradas = obter_resultado(chat.history)
  resultado_Final = chat.send_message(template_resultado(chat.history,num_certas_erradas[0],num_certas_erradas[1]))
  print(borda_quadrada(resultado_Final.text), "\n")
else:
  print("O Gemini não conhece esse tema. Para iniciar o jogo escolha outro tema")


*************************************************
* GeminAIQuiz - Quiz Dinâmico com Google Gemini *
*************************************************
Escolha um tema para o Quiz (Tema sugerido Python): 
python

----------------------------------------------------------------------------------------------------
Qual das seguintes é uma estrutura de dados embutida em Python?

a) Matriz
b) Dicionário
c) Fila
d) Pilha
e) Grafo 

----------------------------------------------------------------------------------------------------
Qual a alternativa correta:
a
----------------------------------------------------------------------------------------------------

 **Errada**

A resposta correta é **b) Dicionário**. Os dicionários são uma estrutura de dados embutida em Python que armazenam pares chave-valor. As matrizes não são uma estrutura de dados embutida em Python; em vez disso, são implementadas usando listas. 

Digite <S> para finalizar o jogo ou qualquer outra tecla para continuar 
 
Qual